# Interfaces con otros lenguajes

A diferencia de **Python**, tanto **Fortran** como **C** y **C++** son lenguajes *compilados*: el código fuente tiene que se convertido a un lenguaje ejecutable para poder ser utilizado. Como mencionamos anteriormente, esto suele ser muy conveniente en el flujo de programación usual pero puede haber casos en que el código en Python es notablemente más lento, o casos en que ya existen rutinas en otros lenguajes y convertirlas a un nuevo lenguaje sea mucho trabajo. En esos casos, una solución posible y conveniente es utilizar un lenguaje compilado para rutinas específicas y Python para todo el resto del trabajo (entrada/salida, manejo de datos, graficación, etc).


## Ejemplo 1: Rotación de vectores

Supongamos que queremos resolver el problema de la rotación de vectores en el espacio usando los [tres ángulos de Euler](https://mathworld.wolfram.com/EulerAngles.html). 

Dado un array con los tres ángulos de Euler, calculamos la matriz de rotación (de 3 x 3), y la aplicamos (esto es, multiplicamos) a un arreglo de _N_ vectores.



En una primera versión, podemos hacerlo en Python directamente en el notebook:

In [1]:
import numpy as np

def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)

In [2]:
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

Si ya tenemos un módulo donde están programadas las funciones necesarias 

In [3]:
# %load rotacion_p.py
#! /usr/bin/ipython3
import numpy as np


def matrix_rotation(angles):
  cx, cy, cz = np.cos(angles)
  sx, sy, sz = np.sin(angles)
  R = np.zeros((3, 3))
  R[0, 0] = cx * cz - sx * cy * sz
  R[0, 1] = cx * sz + sx * cy * cz
  R[0, 2] = sx * sy

  R[1, 0] = -sx * cz - cx * cy * sz
  R[1, 1] = -sx * sz + cx * cy * cz
  R[1, 2] = cx * sy

  R[2, 0] = sy * sz
  R[2, 1] = -sy * cz
  R[2, 2] = cy
  return R


def rotate(angles, v):
  return np.dot(matrix_rotation(angles), v)


es fácil utilizarlas:

In [4]:
N = 100
# Ángulos de Euler
angle = np.random.random(3)
# Definimos N vectores tridimensionales
v = np.random.random((3, N))

In [5]:
y = rotate(angle,v)

In [6]:
print(angle)
print(y[:,0:5].T)

[0.06193349 0.99877501 0.79127934]
[[ 0.90717864  0.68082735 -0.01987772]
 [ 0.43798584  0.78786433  0.54493442]
 [ 0.75536562  0.51102298  0.41541194]
 [ 0.17681573  0.1682188  -0.02342615]
 [ 0.20673471  0.38302061  0.17507386]]


### Importando desde un módulo en un archivo

También podemos tener las funciones para la rotación en un módulo `rotaciones.py`. Recordemos cómo se importarían en este caso

In [7]:
pwd

'/home/fiol/Clases/IntPython/clases-python/clases'

In [8]:
cd ../scripts/interfacing_python

/home/fiol/Clases/IntPython/clases-python/scripts/interfacing_python


In [9]:
ls

__pycache__/  rotaciones.py


In [10]:
import rotaciones as rotp

In [11]:
help(rotp)

Help on module rotaciones:

NAME
    rotaciones

FUNCTIONS
    matrix_rotation(angles)
    
    rotate(angles, v)

FILE
    /home/fiol/Clases/IntPython/clases-python/scripts/interfacing_python/rotaciones.py




In [12]:
yp = rotp.rotate(angle,v)

In [13]:
print(yp[:,0:5].T)
np.allclose(y,yp)

[[ 0.90717864  0.68082735 -0.01987772]
 [ 0.43798584  0.78786433  0.54493442]
 [ 0.75536562  0.51102298  0.41541194]
 [ 0.17681573  0.1682188  -0.02342615]
 [ 0.20673471  0.38302061  0.17507386]]


True

## Interfaces con Fortran

Veamos cómo trabajar si tenemos el código para realizar las rotaciones en Fortran

### Primer ejemplo: Nuestro código

El código en Fortran que tenemos es:

```fortran
function rotate(theta, v, N) result(y)
  implicit none
  integer :: N
  real(8), intent(IN) :: theta(3)
  real(8), intent(IN) :: v(3,N)
  real(8) :: y(3,N)
  real(8) :: R(3,3)
  real(8) :: cx, cy, cz, sx, sy, sz

  ! Senos y Cosenos de los tres ángulos de Euler 
  cx = cos(theta(1)); cy = cos(theta(2)); cz = cos(theta(3))
  sx = sin(theta(1)); sy = sin(theta(2)); sz = sin(theta(3))

  ! Matriz de rotación
  R(1,1) = cx*cz - sx*cy*sz
  R(1,2) = cx*sz + sx*cy*cz
  R(1,3) = sx*sy

  R(2,1) = -sx*cz - cx*cy*sz
  R(2,2) = -sx*sz + cx*cy*cz
  R(2,3) = cx*sy

  R(3,1) = sy*sz
  R(3,2) = -sy*cz
  R(3,3) = cy

  ! Aplicamos la rotación
  y = matmul(R, v)
end function rotate
```

In [14]:
cd ../interfacing_F

/home/fiol/Clases/IntPython/clases-python/scripts/interfacing_F


### F2PY
F2PY  -Fortran to Python interface generator- es una utilidad que permite generar una interface para utilizar funciones y datos definidos en Fortran desde Python.

Información sobre la interfaz entre Fotran y Python, y en particular sobre F2PY, puede encontrarse en:

- [Scipy cookbook](http://scipy-cookbook.readthedocs.io/items/idx_interfacing_with_other_languages.html)
- [F2PY Users Guide and Reference Manual](https://docs.scipy.org/doc/numpy-dev/f2py/index.html)
- [Fortran Best Practices](http://www.fortran90.org/src/best-practices.html#interfacing-with-python)
- http://websrv.cs.umt.edu/isis/index.php/F2py_example

El primer paso es utilizar esta utilidad:
```bash
$ f2py3 -c rotacion.f90 -m rotacion_f
```

In [15]:
!f2py3 -c rotacion.f90 -m rotacion_f

running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "rotacion_f" sources
INFO: f2py options: []
INFO: f2py:> /tmp/tmp8o0oz7vs/src.linux-x86_64-3.11/rotacion_fmodule.c
creating /tmp/tmp8o0oz7vs/src.linux-x86_64-3.11
Reading fortran codes...
	Reading file 'rotacion.f90' (format:free)
Post-processing...
	Block: rotacion_f
			Block: rotaciones
				Block: matrix_rotation
appenddecl: "dimension" not implemented.
				Block: rotate
appenddecl: "dimension" not implemented.
			Block: test_rotation
Applying post-processing hooks...
  character_backward_compatibility_hook
Post-processing (stage 2)...
	Block: rotacion_f
		Block: unknown_interface
			Block: rotaciones
				Block: matrix_rotation
				Block: rotate
			Block: test_rotation
Building modules...
    Buildi

In [16]:
from rotacion_f import rotaciones as rotf

In [17]:
yf = rotf.rotate(angle, v)
print(y[:,0:5].T)
print(yf[:,0:5].T)
np.allclose(yf,y)

[[ 0.90717864  0.68082735 -0.01987772]
 [ 0.43798584  0.78786433  0.54493442]
 [ 0.75536562  0.51102298  0.41541194]
 [ 0.17681573  0.1682188  -0.02342615]
 [ 0.20673471  0.38302061  0.17507386]]
[[ 0.90717864  0.68082735 -0.01987772]
 [ 0.43798584  0.78786433  0.54493442]
 [ 0.75536562  0.51102298  0.41541194]
 [ 0.17681573  0.1682188  -0.02342615]
 [ 0.20673471  0.38302061  0.17507386]]


True

In [18]:
%timeit rotate(angle,v)

6.42 µs ± 107 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [19]:
%timeit rotf.rotate(angle, v)

2.48 µs ± 24.2 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Veamos qué es exactamente lo que importamos:

In [20]:
np.info(rotf.rotate)

y = rotate(angles,v,[n])

Wrapper for ``rotate``.

Parameters
----------
angles : input rank-1 array('d') with bounds (3)
v : input rank-2 array('d') with bounds (3,n)

Other Parameters
----------------
n : input int, optional
    Default: shape(v, 1)

Returns
-------
y : rank-2 array('d') with bounds (3,n) and rotate storage


Como vemos, estamos usando la función `rotate` definida en Fortran. Notar que:
* Tiene tres argumentos.
   * Dos argumentos requeridos: `angles` y `v`
   * Un argumento, correspondiente a la dimensión `n` que F2PY automáticamente detecta como opcional.

             

### Segundo Ejemplo: Código heredado

La conversión que realizamos con f2py3 la podríamos haber realizado en dos pasos:

```bash
$ f2py3 rotacion.f90 -m rotacion_f -h rotacion.pyf
$ f2py3 -c rotacion.pyf -m rotacion_f
```
En el primer paso se crea un archivo *signature* que después se utiliza para crear el módulo que llamaremos desde **Python**. Haciéndolo en dos pasos nos permite modificar el texto del archivo *.pyf* antes de ejecutar el segundo comando.

Esto es útil cuando el código original no es lo suficientemente "moderno", no tiene toda la información necesaria sobre los argumentos o es un código que uno no quiere o puede editar. Veamos que forma tienen con un ejemplo más simple (tomado de la [guía de usuario](https://docs.scipy.org/doc/numpy-dev/f2py/getting-started.html)):

```fortran
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
```

In [21]:
!f2py3 --overwrite-signature fib1.f -m fib1 -h fib1.pyf

Reading fortran codes...
	Reading file 'fib1.f' (format:fix,strict)
Post-processing...
	Block: fib1
			Block: fib
Applying post-processing hooks...
  character_backward_compatibility_hook
Post-processing (stage 2)...
Saving signatures to file "./fib1.pyf"


El contenido del archivo `fib1.pyf` es:

```fortran90
python module fib1 ! in 
    interface  ! in :fib1
        subroutine fib(a,n) ! in :fib1:fib1.f
            real*8 dimension(n) :: a
            integer, optional,check(len(a)>=n),depend(a) :: n=len(a)
        end subroutine fib
    end interface 
end python module fib1
```

Este código indica que tenemos una subrutina que toma dos argumentos:
- `a` es un array
- `n` es un entero opcional, que tiene que ser mayor que `len(a)`

In [22]:
!f2py3  -c fib1.pyf fib1.f 

running build
running config_cc
INFO: unifing config_cc, config, build_clib, build_ext, build commands --compiler options
running config_fc
INFO: unifing config_fc, config, build_clib, build_ext, build commands --fcompiler options
running build_src
INFO: build_src
INFO: building extension "fib1" sources
creating /tmp/tmpvzr316vv/src.linux-x86_64-3.11
INFO: f2py options: []
INFO: f2py: fib1.pyf
Reading fortran codes...
	Reading file 'fib1.pyf' (format:free)
Post-processing...
	Block: fib1
			Block: fib
Applying post-processing hooks...
  character_backward_compatibility_hook
Post-processing (stage 2)...
Building modules...
    Building module "fib1"...
    Generating possibly empty wrappers"
    Maybe empty "fib1-f2pywrappers.f"
        Constructing wrapper function "fib"...
          fib(a,[n])
    Wrote C/API module "fib1" to file "/tmp/tmpvzr316vv/src.linux-x86_64-3.11/fib1module.c"
INFO:   adding '/tmp/tmpvzr316vv/src.linux-x86_64-3.11/fortranobject.c' to sources.
INFO:   adding '/t

In [24]:
ls -ltr

total 780
drwxr-xr-x. 1 fiol fiol     16 mar 27  2023 fib2.cpython-39-darwin.so.dSYM/
drwxr-xr-x. 1 fiol fiol     16 mar 27  2023 fib3.cpython-39-darwin.so.dSYM/
-rwxr-xr-x. 1 fiol fiol  55448 mar 27  2023 fib3.cpython-39-darwin.so*
-rw-r--r--. 1 fiol fiol    403 mar 27  2023 fib2.pyf
-rwxr-xr-x. 1 fiol fiol  55448 mar 27  2023 fib2.cpython-39-darwin.so*
-rw-r--r--. 1 fiol fiol    347 mar 27  2023 fib1.f
-rwxr-xr-x. 1 fiol fiol  55512 mar 27  2023 fib1.cpython-39-darwin.so*
-rw-r--r--. 1 fiol fiol   1007 mar 27  2023 rotacion.pyf
-rwxr-xr-x. 1 fiol fiol  58480 mar 27  2023 rotacion_f.cpython-39-darwin.so*
-rw-r--r--. 1 fiol fiol   2202 mar 27  2023 rotacion.f90
-rw-r--r--. 1 fiol fiol    558 mar 27  2023 rotaciones.mod
-rw-r--r--. 1 fiol fiol    372 mar 27  2023 fib3.f
-rwxr-xr-x. 1 fiol fiol 124216 mar 22 13:27 fib2.cpython-311-x86_64-linux-gnu.so*
-rwxr-xr-x. 1 fiol fiol 124216 mar 22 13:28 fib3.cpython-311-x86_64-linux-gnu.so*
-rwxr-xr-x. 1 fiol fiol 151944 mar 22 14:28 rotacion_f.c

In [25]:
import fib1

In [26]:
print(fib1.fib.__doc__)

fib(a,[n])

Wrapper for ``fib``.

Parameters
----------
a : input rank-1 array('d') with bounds (n)

Other Parameters
----------------
n : input int, optional
    Default: shape(a, 0)



In [27]:
a = np.zeros(12)
fib1.fib(a)
print(a)

[ 0.  1.  1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


Podemos darle además a la función `fib` el argumento opcional `n`. Solo que debe cumplir la condición especificada en el archivo *fib1.pyf*

In [28]:
a = np.zeros(12)
fib1.fib(a,12)
print(a)

[ 0.  1.  1.  2.  3.  5.  8. 13. 21. 34. 55. 89.]


In [29]:
a = np.zeros(12)
fib1.fib(a,8)
print(a)

error: (shape(a, 0) == n) failed for 1st keyword n: fib:n=8

In [30]:
a = np.zeros(12)
fib1.fib(a,18)
print(a)

error: (shape(a, 0) == n) failed for 1st keyword n: fib:n=18

Esta es una de las características de F2PY: hace un chequeo básico de los argumentos. Hay otro error que no llega a atrapar: Si le pasamos un array que no es del tipo indicado, falla (sin avisar). Éste claramente no es el comportamiento deseado:

In [31]:
a = np.zeros(12, dtype=int)
fib1.fib(a)
print(a)

[0 0 0 0 0 0 0 0 0 0 0 0]


Vamos a modificar el archivo de *signature* para enseñarle dos cosas:

- El entero es un argumento de entrada (requerido)
- El *array* `a` es un archivo de salida **exclusivamente**. Entonces no debemos dárselo. La parte `dimension(n)` y `depend(n)` indica que debe crear un vector de ese tamaño.


In [32]:
!f2py3  -c fib2.pyf fib1.f 1&2> /dev/null

In [33]:
import fib2
print(fib2.fib.__doc__)

a = fib(n)

Wrapper for ``fib``.

Parameters
----------
n : input int

Returns
-------
a : rank-1 array('d') with bounds (n)



In [34]:
fib2.fib(9)

array([ 0.,  1.,  1.,  2.,  3.,  5.,  8., 13., 21.])

In [35]:
print(fib2.fib(14))

[  0.   1.   1.   2.   3.   5.   8.  13.  21.  34.  55.  89. 144. 233.]


La segunda manera de arreglar este problema, en lugar de modificar el archivo de *signature* podría haber sido modificar el código (o hacer una rutina intermedia). Agregando comentarios de la forma `Cf2py` no influimos en la compilación `Fortran` pero F2PY los reconoce. En este caso le damos la información sobre la intención de los argumentos en el código:

```fortran
      SUBROUTINE FIB(A,N)
C
C     CALCULATE FIRST N FIBONACCI NUMBERS
C
      INTEGER N
      REAL*8 A(N)
Cf2py intent(in) n
Cf2py intent(out) a
Cf2py depend(n) a
      DO I=1,N
         IF (I.EQ.1) THEN
            A(I) = 0.0D0
         ELSEIF (I.EQ.2) THEN
            A(I) = 1.0D0
         ELSE 
            A(I) = A(I-1) + A(I-2)
         ENDIF
      ENDDO
      END
```

In [36]:
!f2py3  -c fib3.f -m fib3 1&2> /dev/null

In [37]:
import fib3
print(fib3.fib.__doc__)

a = fib(n)

Wrapper for ``fib``.

Parameters
----------
n : input int

Returns
-------
a : rank-1 array('d') with bounds (n)



In [38]:
print(fib2.fib.__doc__)

a = fib(n)

Wrapper for ``fib``.

Parameters
----------
n : input int

Returns
-------
a : rank-1 array('d') with bounds (n)



como vemos, son exactamente iguales.

### F2PY para código en C

Es posible usar F2PY para código escrito en C, pero en ese caso debemos escribir el *signature file* a mano.

Para código en C es conveniente utilizar **Cython**. Cython es un lenguaje de programación pensado para hacer más fácil escribir extensiones a Python en C. Uno escribe el código en Cython, y luego es traducido a C, con optimizaciones.

**Cython** también puede utilizarse con Fortran de una manera similar a cómo se usa con C. Para más información ver la [documentación oficial](http://docs.cython.org/en/latest/index.html)
